# Section F.4 (Experiment 5): Exploration of optimal and near-optimal solutions for a small instance

In [ ]:
from gurobipy import *
import numpy as np

We consider the problem setup in Section F.4.

In [ ]:
W = 200
w = [3,5,7,10,17,22,30,50]
nDemands = 8
d = [1200,1000,1000,400,500,400,600,200]

The following cell solves the subproblem in the column generation approach to the cutting stock problem.

In [ ]:
m_sub = Model()
a = m_sub.addVars(nDemands, vtype = GRB.INTEGER)

def separation(p_val):
    m_sub.addConstr( sum(w[i] * a[i] for i in range(nDemands)) <= W)
    m_sub.setObjective( sum(p_val[i] * a[i] for i in range(nDemands)), GRB.MAXIMIZE)
    m_sub.update()
    m_sub.optimize()
    
    a_val = [a[i].x for i in range(nDemands)]
    sub_obj = m_sub.objval
    
    return sub_obj, a_val

We can now create our constraint generation algorithm.

In [ ]:
# Reformulate the dual problem. 
m_dual = Model()

m_dual.Params.OutputFlag = 0

p = m_dual.addVars(nDemands)

m_dual.setObjective(sum(d[i] * p[i] for i in range(nDemands)), GRB.MAXIMIZE)

# Let us initialize the set of patterns with just those that consist only of a single demand type.
# We can enumerate that set as follows:

patterns = [ [1 for i in range(nDemands)] ]
print(patterns)

nPatterns = len(patterns)

for j in range(nPatterns):
    m_dual.addConstr( sum( patterns[j][i] * p[i] for i in range(nDemands)) <= 1)
    
m_dual.update()
m_dual.optimize()



dual_obj = m_dual.objval

CG_iter = 0

print("Iteration ", CG_iter, ": ", dual_obj)


p_val = [p[i].x for i in range(nDemands)]

sub_objval, a_val = separation(p_val)

tol_eps = 1e-5

isOptimal = sub_objval <= 1 + tol_eps

while (not isOptimal):
    # Add a_val to patterns, and add the constraint to the dual.
    patterns.append(a_val)
    m_dual.addConstr( sum( a_val[i] * p[i] for i in range(nDemands)) <= 1)
    
    # Update the problem and solve it again.
    m_dual.update()
    m_dual.optimize()
    dual_obj = m_dual.objval
    p_val = [p[i].x for i in range(nDemands)]
    
    CG_iter += 1 
    print()
    print("Iteration ", CG_iter, ": ", dual_obj)
    
    # Solve separation problem, and update the isOptimal flag.
    sub_objval, a_val = separation(p_val)
    isOptimal = sub_objval <= 1 + tol_eps
    print("sub_objval = ", sub_objval)
    print("constraint violation = ", 1 - sub_objval),
    print("a_val = ", a_val)
    print("isOptimal = ", isOptimal)
    

To figure out the solution to the original problem, we formulate the primal and solve again.

In [ ]:
nPatterns = len(patterns)

m = Model()
x = m.addVars(nPatterns)

for i in range(nDemands):
    m.addConstr( sum( patterns[j][i] * x[j] for j in range(nPatterns) ) >= d[i])
    
m.setObjective( sum(x[j] for j in range(nPatterns)), GRB.MINIMIZE)

m.update()

m.optimize()

x_vals = [ x[j].x for j in range(nPatterns)]

print("Optimal solution: ", x_vals)

nonzero_pattern_indices = [j for j in range(nPatterns) if x_vals[j] > 0]

for j in nonzero_pattern_indices:
    print("Pattern ", j, ": ", patterns[j], " -- cut ", x_vals[j], " rolls")
    

Now we have found the eight columns described in Section F.4

The following function samples a column.

In [ ]:
from random import sample

min_w = min(w)

def random_a():
    a_val = np.zeros(nDemands)
    remaining_width = W
    valid_widths = [i for i in range(nDemands) if w[i] <= remaining_width]
    while (len(valid_widths) > 0):
        i_random = np.random.choice(valid_widths, 1)
        i_random = i_random[0]
        a_val[i_random] += 1
        remaining_width -= w[i_random]
        valid_widths = [i for i in range(nDemands) if w[i] <= remaining_width]
        
    return a_val

We run the column randomization method 20000 times and investigate the returned optimal solutions and nearly-optimal solutions.

In [ ]:
nSimulations = 20000
pattern_sets = []
obj_vec = np.zeros(nSimulations)
eps = np.finfo(float).eps

# save information about the optimal basis from the sampled patterns
nOpt = 0
Opt_pattern_sets = []

# save information about the nearly-optimal basis from the sampled patterns
nNearOpt = 0
NearOpt_pattern_sets = []

for s in range(nSimulations):
    # Sample 100 random patterns:
    nPatterns = 100
    patterns = []
    for j in range(nPatterns):
        patterns.append( random_a() )

    # Solve the primal problem to see which patterns are involved:
    m = Model()
    x = m.addVars(nPatterns)

    m.Params.OutputFlag = 0
    m.Params.Method = 0

    for i in range(nDemands):
        m.addConstr( sum( patterns[j][i] * x[j] for j in range(nPatterns) ) >= d[i])
    m.setObjective( sum(x[j] for j in range(nPatterns)), GRB.MINIMIZE)

    m.update()
    m.optimize()

    x_vals = [ x[j].x for j in range(nPatterns)]
    nonzero_pattern_indices = [j for j in range(nPatterns) if x[j].vbasis == 0]
    
    #print("Objective value: ", m.objval)
    #for j in nonzero_pattern_indices:
    #    print("Pattern ", j, ": ", patterns[j], " -- cut ", x_vals[j], " rolls")
    
    used_patterns = [patterns[j] for j in nonzero_pattern_indices]
    
    pattern_sets.append(used_patterns)
    obj_vec[s] = m.objval

    # optimal solutions
    if m.objval <= 324.5 + eps:
        nOpt += 1
        Opt_pattern_sets.append(used_patterns)
    
    # nearly optimal solutions
    if m.objval <= 324.5 + 2.0:
        nNearOpt += 1
        NearOpt_pattern_sets.append(used_patterns)


In [ ]:
# Number of times that the column randomization method gives the optimal solution:
print("Number of optimal basis = ",nOpt)
# Number of unique patterns (columns) in the collection of all sampled optimal basis:
uniq_pattern = [Opt_pattern_sets[i][j] for i in range(nOpt) for j in range(len(Opt_pattern_sets[i]))]
uniq_pattern_tuple = set([tuple(i) for i in uniq_pattern])
print("Number of unqiue patterns among the sampled optimal basis = ",len(uniq_pattern_tuple))
# Counting the incidence of each pattern:
uniq_pattern2 = list(uniq_pattern_tuple)
incidence = np.zeros(len(uniq_pattern2))

for t in range(len(uniq_pattern2)):
    incidence[t] = sum( [ tuple(i) == uniq_pattern2[t] for i in uniq_pattern ])

print("Maximum Incidence = ",incidence.max())
print("Average Incidence = ",incidence.mean())

In [ ]:
# Number of times that the column randomization method gives the nearly-optimal solution:
print("Number of nearly-optimal basis = ",nNearOpt)
# Number of unique patterns (columns) in the collection of all sampled nearly-optimal basis:
uniq_pattern_nearopt = [NearOpt_pattern_sets[i][j] for i in range(nNearOpt) for j in range(len(NearOpt_pattern_sets[i]))]
uniq_pattern_tuple_nearopt = set([tuple(i) for i in uniq_pattern_nearopt])
print("Number of unqiue patterns among the sampled nearly-optimal basis = ",len(uniq_pattern_tuple_nearopt))
# Counting the incidence of each pattern:
uniq_pattern2_nearopt = list(uniq_pattern_tuple_nearopt)
incidence_nearopt = np.zeros(len(uniq_pattern2_nearopt))

for t in range(len(uniq_pattern2_nearopt)):
    incidence_nearopt[t] = sum( [ tuple(i) == uniq_pattern2_nearopt[t] for i in uniq_pattern_nearopt])

print("Maximum Incidence = ",incidence_nearopt.max())
print("Average Incidence = ",incidence_nearopt.mean())